### Model training

The filtered, calculated and encoded features can now be trained with appropriate models.

The following approaches are considered:
- Multi classification problem with clustered data
- Mutli regression problem with two outputs (longitude/latitude)

In terms of the data we have the following approaches:
- variable sequence length - can take all points in POLYLINE in consideration, mask sequence if necessary
- fixed sequence length - take 10 points from beginning of POLYLINE and 10 points from end of polyline 


Algorithms:
- Long term short term NN (multi-class classification and regression)
    - able to handle variable sequence length, therefore the total trip POLYLINE can be used
- Random forest(regression and classification) 
    - can handle outliers well as dataset still contains outliers
    - runs efficiently on large data set
    
Metrics:
- Classification of clusters: AUC + Avg distance of last point to cluster center
- Regression: MAPE + Avg distance of last point to cluster center


In [190]:
import os,sys
import pandas as pd 
import numpy as np 
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from model import MultioutputModel

In [7]:
is_sagemaker_notebook = False

In [8]:
if is_sagemaker_notebook:
    prefix = "/home/ec2-user/SageMaker"
else:
        prefix = "/Users/Q619505/PycharmProjects/personal-projects/taxi-destination-prediction"

utils_path = os.path.join(f'{prefix}/src/utils/')
pp_path = os.path.join(f'{prefix}/src/preprocessing/')
model_path = os.path.join(f'{prefix}/src/modelling/')

if utils_path not in sys.path:
    sys.path.append(utils_path)

if pp_path not in sys.path:
    sys.path.append(pp_path)

if model_path not in sys.path:
    sys.path.append(model_path)

In [149]:
from data_cleaning import create_fix_length_sequences, split_lat_lon
from geo_spatial import convert_polyline_to_geojson_format, convert_string_to_geojson, haversine_distance
from model import *

In [15]:
if is_sagemaker_notebook:
    train_data = pd.read_parquet(f's3://think-tank-casestudy/features_engineered/n_cluster_{n_cluster}/feature_engineered_train.parquet')
    test_data = pd.read_parquet(f's3://think-tank-casestudy/features_engineered/n_cluster_{n_cluster}/feature_engineered_test.parquet')
else:
    train_data = pd.read_csv(f'{prefix}/data/processed/train_data_encoded.csv', header=0,index_col=False)
    test_data = pd.read_csv(f'{prefix}/data/processed/test_data_encoded.csv', header=0, index_col=False)
    

Limit to only necessary attribtues

In [159]:
X_train = train_data[['trip_id', 'final_point', 'polyline', 'n_coordinate_points', 'total_distance_km','total_flight_time_minutes']]
X_test = test_data[['trip_id', 'final_point','polyline',  'n_coordinate_points', 'total_distance_km','total_flight_time_minutes']]

In [17]:
X_train = convert_polyline_to_geojson_format(X_train, 'polyline').rename(columns={'polyline': 'sequence'})
X_test = convert_polyline_to_geojson_format(X_test, 'polyline').rename(columns={'polyline': 'sequence'})

In [51]:
def create_fix_length_sequences(data, n_limited,  sequence_column: str) -> pd.DataFrame:
    """
    Selects first n_limited from array and last n_limited from array and concatenates them 
    """
    modified_data = data.copy()
    modified_data[sequence_column + '_transformed'] = modified_data[sequence_column]\
    .apply(lambda sequence: sequence[0 : n_limited] + sequence[-1 * (n_limited+1): -1])

    return modified_data

In [54]:
X_train = create_fix_length_sequences(X_train, 8, 'sequence')
X_test = create_fix_length_sequences(X_test, 8, 'sequence')

In [101]:
def pair_wise_haversine_distance(point_sequence :list) -> list:
    """haversine distance between points pair wise 
    """
    distance_array = []
    prev_point = None
    for current_point in point_sequence:
        if prev_point != None:
            current_distance = haversine_distance(lat1=prev_point[0],lat2=current_point[0], lon1=prev_point[1],lon2=current_point[1])
            distance_array = np.append(current_distance, distance_array)
        prev_point = current_point
    return distance_array

In [103]:
X_train['distance_sequence'] = X_train.sequence_transformed.apply(lambda sequence_: pair_wise_haversine_distance(sequence_))
X_test['distance_sequence'] = X_test.sequence_transformed.apply(lambda sequence_: pair_wise_haversine_distance(sequence_))

In [105]:
X_train, y_train = X_train[['distance_sequence']], X_train[['final_point']]
X_test, y_test = X_test[['distance_sequence']], X_test[['final_point']]

In [160]:
y_train = X_train[['final_point']]
y_test = X_test[['final_point']]

In [143]:
X_train = pd.DataFrame(sequence_ for sequence_ in X_train.distance_sequence).add_prefix('x_')
X_test = pd.DataFrame(sequence_ for sequence_ in X_test.distance_sequence).add_prefix('x_')

In [146]:
X_train.to_csv('X_train_distance_sequence.csv', header=True, index=False)
X_test.to_csv('X_test_distance_sequence.csv', header=True, index=False)

In [161]:
y_train = convert_polyline_to_geojson_format(y_train, 'final_point')

In [162]:
y_train = split_lat_lon(y_train, 'final_point')

In [166]:
y_train.lon_final_point = y_train.lon_final_point.astype(float)
y_train.lat_final_point = y_train.lat_final_point.astype(float)

In [168]:
y_train = y_train.drop(['final_point'], axis=1)

In [169]:
y_test = convert_polyline_to_geojson_format(y_test, 'final_point')

In [170]:
y_test = split_lat_lon(y_test, 'final_point')

In [171]:
y_test.lon_final_point = y_test.lon_final_point.astype(float)
y_test.lat_final_point = y_test.lat_final_point.astype(float)

In [172]:
y_test = y_test.drop(['final_point'], axis=1)

In [174]:
y_train.to_csv('y_train_distance_sequence.csv', header=True, index=False)
y_test.to_csv('y_test_distance_sequence.csv', header=True, index=False)

### 1a) Cluster Prediction - Multi class classifier Random forest

In [42]:
n_cluster = 4000

In [43]:
non_feature_columns_config_1_list = ['trip_id', 'total_distance_km', 'total_flight_time_minutes', 
'start_point_lon', 'start_point_lat', 'dest_point_lon', 'dest_point_lat', 'cluster_label', 'sequence', 'final_point', 'polyline',
                                    'center_lon', 'index', 'center_lat', 'day_type']

In [44]:
features_config_1 = [column_ for column_ in train_data.columns if not column_ in non_feature_columns_config_1_list]

In [45]:
label_config_1 = ['cluster_label']

In [46]:
train_data_reduced = train_data.sample(20000, axis=0)

In [47]:
#train_data = create_fix_length_sequences(train_data, 10)
#test_data = create_fix_length_sequences(test_data, 10)

In [48]:
X_train = train_data_reduced[features_config_1].to_numpy()
y_train = train_data_reduced[label_config_1]

In [49]:
X_test = test_data[features_config_1].to_numpy()
y_test = test_data[label_config_1]

In [50]:
#start_sequence_train = pd.DataFrame(train_data_reduced.START_SEQUENCE.tolist()).to_numpy()
#stop_sequence_train = pd.DataFrame(train_data_reduced.STOP_SEQUENCE.tolist()).to_numpy()

#start_sequence_test = pd.DataFrame(test_data.START_SEQUENCE.tolist()).to_numpy()
#stop_sequence_test = pd.DataFrame(test_data.STOP_SEQUENCE.tolist()).to_numpy()

In [51]:
#X_train = np.concatenate((X_train, start_sequence_train, stop_sequence_train), axis=1).astype(float)
#X_test = np.concatenate((X_test, start_sequence_test, stop_sequence_test), axis=1).astype(float)

In [52]:
assert(X_train.shape[1] == X_test.shape[1])

In [53]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(max_depth=20, random_state=0)
clf.fit(X_train, y_train)

/Users/Q619505/Library/Caches/pypoetry/virtualenvs/taxi-destination-prediction-tbwwnVrE-py3.11/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(max_depth=20, random_state=0)

In [54]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(min_samples_leaf=0.05, random_state=0)
regr.fit(X_train, y_train)

/Users/Q619505/Library/Caches/pypoetry/virtualenvs/taxi-destination-prediction-tbwwnVrE-py3.11/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(min_samples_leaf=0.05, random_state=0)

In [55]:
y_pred = regr.predict(X_test)

In [56]:
df_pred = pd.DataFrame(y_pred, columns = ['LON','LAT'])

ValueError: Shape of passed values is (262, 1), indices imply (262, 2)

In [ ]:
df_pred = pd.concat([y_test, df_pred],axis=1)

In [ ]:
df_pred['h_distance'] = df_pred.apply(lambda row: haversine_distance(lat1=row.DEST_POINT_LAT,
                                                                     lat2=row.LAT,
                                                                     lon1=row.DEST_POINT_LON,
                                                                     lon2=row.LON), axis=1)

In [ ]:
df_pred.h_distance.describe()

count    269.000000
mean       2.103287
std        7.648067
min        0.036385
25%        0.667274
50%        1.035733
75%        1.487928
max      114.168664
Name: h_distance, dtype: float64

Convert sequence of points (polyline) to geo json format 

In [32]:
train_data = convert_polyline_to_geojson_format(data=train_data, name_column='polyline')
test_data = convert_polyline_to_geojson_format(data=test_data, name_column='polyline')

In [33]:
train_data['start_point'] = train_data.polyline.apply(lambda value_ : value_[0])
test_data['start_point'] = test_data.polyline.apply(lambda value_ : value_[0])

Create two fixed length sequences
- first 8 points
- last 8 points (excluding final point)

In [34]:
train_data = create_fix_length_sequences(train_data, 8, start_sequence_column='start_sequence', stop_sequence_column='dest_sequence', sequence_column='polyline')
test_data = create_fix_length_sequences(test_data, 8,start_sequence_column='start_sequence', stop_sequence_column='dest_sequence', sequence_column='polyline')

In [12]:
#def _normalize_coordinates(coordinates):
#    return (coordinates[0] + 90)/ 180, (coordinates[1] + 180)/ 360

In [37]:
train_data[train_data.n_coordinate_points == 10].final_point[26]

'[-8.619993, 41.146839]'

In [39]:
len(train_data[train_data.n_coordinate_points == 10].dest_sequence[26])

7

In [14]:
train_data['dest_point_lat_norm'] = train_data['final_point']\
.apply(lambda coordinates: _normalize_coordinates(convert_string_to_geojson(coordinates))[0]) 
train_data['dest_point_lon_norm'] = train_data['final_point']\
.apply(lambda coordinates: _normalize_coordinates(convert_string_to_geojson(coordinates))[1]) 

In [15]:
test_data['dest_point_lat_norm'] = test_data['final_point']\
.apply(lambda coordinates: _normalize_coordinates(convert_string_to_geojson(coordinates))[0]) 
test_data['dest_point_lon_norm'] = test_data['final_point']\
.apply(lambda coordinates: _normalize_coordinates(convert_string_to_geojson(coordinates))[1]) 

In [20]:
train_data.normalized_polyline = train_data.normalized_polyline.apply(lambda sequence : sequence.reshape((1,10,2)))
test_data.normalized_polyline = test_data.normalized_polyline.apply(lambda sequence : sequence.reshape((1,10,2)))

In [28]:
X_train = train_data[['normalized_polyline']]
X_test = test_data[['normalized_polyline']]
y_train = train_data[['dest_point_lat_norm', 'dest_point_lon_norm']]
y_test =  test_data[['dest_point_lat_norm', 'dest_point_lon_norm']]

In [35]:
X_train.to_csv(f'{prefix}/ml-project-taxi-prediction/data/processed/X_train.csv', header=True, index=False)
X_test.to_csv(f'{prefix}/ml-project-taxi-prediction/data/processed/X_test.csv', header=True, index=False)
y_train.to_csv(f'{prefix}/ml-project-taxi-prediction/data/processed/y_train.csv', header=True, index=False)
y_test.to_csv(f'{prefix}/ml-project-taxi-prediction/data/processed/y_test.csv', header=True, index=False)

OSError: Cannot save file into a non-existent directory: '/Users/Q619505/PycharmProjects/personal-projects/taxi-destination-prediction/ml-project-taxi-prediction/data/processed'

###  2) NN with LSTM - full sequence only - no tabular data

In [ ]:
features_config = ['SEQUENCE']
label_config = ['CLUSTER_LABEL']

In [ ]:
X_train = train_data[features_config]
y_train = train_data[label_config]
X_test = test_data[features_config]
y_test = test_data[label_config]

In [ ]:
#max length of sequence
if train_data.N_COORDINATE_POINTS.max() > test_data.N_COORDINATE_POINTS.max():
    max_sequence= train_data.N_COORDINATE_POINTS.max()
else:
    max_sequence = test_data.N_COORDINATE_POINTS.max()

In [ ]:
def create_lstm_shape(sequence_,max_sequence):
    sequence_array = np.array(sequence_)
    zeros_ = np.zeros(2*max_sequence-len(sequence_array))
    return np.append(sequence_array,zeros_).reshape(1,max_sequence,2)

In [ ]:
X_test['SEQUENCE'] = X_test.SEQUENCE.apply(lambda sequence_: create_lstm_shape(sequence_,max_sequence))
X_train['SEQUENCE'] = X_train.SEQUENCE.apply(lambda sequence_: create_lstm_shape(sequence_,max_sequence))

In [ ]:
lstm_model = Sequential()
lstm_model.add(tensorflow.compat.v1.keras.layers.CuDNNLSTM(200, input_shape=(612,2)))
lstm_model.add(Dense(4000, activation='softmax'))
print(lstm_model.summary())

In [ ]:
lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['AUC'])

In [ ]:
X_train = np.array([np.array(val) for val in X_train])

In [ ]:
X_train

In [ ]:
history=lstm_model.fit(X_train,
                       y_train.astype(np.float32), 
                       validation_split=0.2, epochs=100, batch_size=448, verbose=2)

### 2b) NN - fixed sequence

In [ ]:
df_sequence_train_start = pd.DataFrame(train_data.START_SEQUENCE.tolist()).fillna(2000)
df_sequence_train_stop = pd.DataFrame(train_data.STOP_SEQUENCE.tolist()).fillna(2000)

df_sequence_test_start = pd.DataFrame(test_data.START_SEQUENCE.tolist()).fillna(2000)
df_sequence_test_stop = pd.DataFrame(test_data.STOP_SEQUENCE.tolist()).fillna(2000)

In [ ]:
df_sequence_train_start.shape

In [ ]:
#fill nas with arbitrary large number to mask later
#df_sequence_train = pd.DataFrame(train_data.SEQUENCE.tolist()).fillna(2000).to_numpy()
#df_sequence_test = pd.DataFrame(test_data.SEQUENCE.tolist()).fillna(2000).to_numpy()

In [ ]:
features_config_2 = ['N_COORDINATE_POINTS','TOTAL_DISTANCE_KM','2013_10',
       '2013_11', '2013_12', '2013_7', '2013_8', '2013_9', '2014_1', '2014_2',
       '2014_3', '2014_4', '2014_5', '2014_6', '2014_7', '10.0', '12.0',
       '13.0', '14.0', '15.0', '18.0', '20.0', '21.0', '23.0', '25.0', '26.0',
       '27.0', '28.0', '33.0', '34.0', '35.0', '36.0', '38.0', '40.0', '42.0',
       '52.0', '53.0', '54.0', '56.0', '57.0', '58.0', '6.0', '60.0', '61.0',
       '63.0', '7.0', '9.0', 'OTHER', 'Cloudy', 'Foggy', 'Rainy', 'Sunny',
       'Windy', 'A', 'B', 'C', '16.0', '2014_10', '2014_11', '2014_12',
       '2014_8', '2014_9', '47.0', '49.0']
label_config_2 = ['CLUSTER_LABEL']

In [ ]:
X_train = train_data[features_config_2].astype(float)
X_test = test_data[features_config_2].astype(float)

In [ ]:
X_train = pd.concat([df_sequence_train_start,df_sequence_train_stop, X_train], axis=1)
X_test = pd.concat([df_sequence_test_start,df_sequence_test_stop, X_test], axis=1)

In [ ]:
y_train = train_data[label_config_2]
y_test =  test_data[label_config_2]

In [ ]:
model = Sequential()
model.add(Input(shape=(X_train.shape[1])))
model.add(tensorflow.keras.layers.Masking(mask_value=2000))
model.add(LSTM(200, activation='relu'))
model.add(Dense(4000, activation='softmax'))
print(model.summary())

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['AUC'])
          
#X_pred_train = model.fit(X_train)